In [37]:
'''
  code by Tae Hwan Jung(Jeff Jung) @graykode
  参考了 理解Pytorch中LSTM的输入输出参数含义  AutoML机器学习
'''
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

dtype = torch.FloatTensor

char_arr = [c for c in 'abcdefghijklmnopqrstuvwxyz'] #输出一个每个字母的list
word_dict = {n: i for i, n in enumerate(char_arr)} #word->index
number_dict = {i: w for i, w in enumerate(char_arr)}#index->word
n_class = len(word_dict) # number of class(=number of vocab) #分类长度26

seq_data = ['make', 'need', 'coal', 'word', 'love', 'hate', 'live', 'home', 'hash', 'star']

# TextLSTM Parameters
n_step = 3  
n_hidden = 8 #单词向量维度

def make_batch(seq_data):
    input_batch, target_batch = [], []

    for seq in seq_data:
        input = [word_dict[n] for n in seq[:-1]] # 'm', 'a' , 'k' is input
        target = word_dict[seq[-1]] # 'e' is target
        input_batch.append(np.eye(n_class)[input]) #独热
        target_batch.append(target)
    print('input_batch被处理为的形状是:',Variable(torch.Tensor(input_batch)).shape) #输出一下形状,这里的顺序是(batch,输入维度,输出维度)
    return Variable(torch.Tensor(input_batch)), Variable(torch.LongTensor(target_batch))
flag=False
class TextLSTM(nn.Module):
    def __init__(self):
        super(TextLSTM, self).__init__()

        self.lstm = nn.LSTM(input_size=n_class, hidden_size=n_hidden,) 
        '''input_size – 输入数据的维度
hidden_size – 隐藏层的大小（即隐藏层节点数量），输出向量的维度等于隐藏节点数
num_layers – 循环层数目,默认为1
bias – 默认为True,
batch_first – 默认为False，是否把batch放在第一维
dropout – 默认为0,如果非0，就在除了输出层插入Dropout。
bidirectional – 是否为双向LSTM'''

        self.W = nn.Parameter(torch.randn([n_hidden, n_class]).type(dtype))
        self.b = nn.Parameter(torch.randn([n_class]).type(dtype)) #这两层为MLP
    flag1 =False
    def forward(self, X):
        input = X.transpose(0, 1)  # X : [n_step, batch_size, n_class] 交换维度的数据

        hidden_state = Variable(torch.zeros(1, len(X), n_hidden))   # [num_layers(=1) * num_directions(=1), batch_size, n_hidden]
        cell_state = Variable(torch.zeros(1, len(X), n_hidden))     # [num_layers(=1) * num_directions(=1), batch_size, n_hidden]
        if self.flag1==False:
            print('input.shape:',input.shape,'RNN输入维度,batch,输出维度','hidden_state.shape:',hidden_state.shape,'(RNN层数*深度, batch, 隐藏层维度)')
            self.flag1=True
        '''输入数据需要按如下形式传入 input, (h_0,c_0)
         LSTM的参数:input: 输入数据，即上面例子中的一个句子（或者一个batch的句子），其维度形状为 (seq_len, batch, input_size)
seq_len: 句子长度，即单词数量，这个是需要固定的。当然假如你的一个句子中只有2个单词，但是要求输入10个单词，这个时候可以用torch.nn.utils.rnn.pack_padded_sequence()或者torch.nn.utils.rnn.pack_sequence()来对句子进行填充或者截断。
batch：就是你一次传入的句子的数量
input_size: 每个单词向量的长度，这个必须和你前面定义的网络结构保持一致

h_0：维度形状为 (num_layers * num_directions, batch, hidden_size):
第一个参数的含义num_layers * num_directions， 即LSTM的层数乘以方向数量。这个方向数量是由前面介绍的bidirectional决定，如果为False,则等于1；反之等于2。
batch：同上
hidden_size: 隐藏层节点数

c_0：维度形状为 (num_layers * num_directions, batch, hidden_size),各参数含义和h_0类似。'''

        outputs, (_ , _ ) = self.lstm(input, (hidden_state, cell_state))
        
        ''''output：维度和输入数据类似，只不过最后的feature部分会有点不同，即 (seq_len, batch, num_directions * hidden_size)
这个输出tensor包含了LSTM模型最后一层每个time step的输出特征，比如说LSTM有两层，那么最后输出的是,表示第二层LSTM每个time step对应的输出。
另外如果前面你对输入数据使用了torch.nn.utils.rnn.PackedSequence,那么输出也会做同样的操作编程packed sequence。
对于unpacked情况，我们可以对输出做如下处理来对方向作分离output.view(seq_len, batch, num_directions, hidden_size), 其中前向和后向分别用0和1表示Similarly, the directions can be separated in the packed case'''
        
        global flag
        if flag==False:
            
            print('outputs[0]:\n\n',outputs[0])
            print('batch1[0]:\n\n',batch1[0])
            print('num_directions1[0]:\n\n',num_directions1[0])
            flag=True
        outputs = outputs[-1]  # [batch_size, n_hidden]
        model = torch.mm(outputs, self.W) + self.b  # model : [batch_size, n_class]
        return model

input_batch, target_batch = make_batch(seq_data)


input_batch被处理为的形状是: torch.Size([10, 3, 26])


In [38]:

model = TextLSTM()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

output = model(input_batch)

# Training
for epoch in range(1000):
    optimizer.zero_grad()

    output = model(input_batch)
    loss = criterion(output, target_batch)
    if (epoch + 1) % 100 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

    loss.backward()
    optimizer.step()

inputs = [sen[:3] for sen in seq_data]

predict = model(input_batch).data.max(1, keepdim=True)[1]
print(inputs, '->', [number_dict[n.item()] for n in predict.squeeze()])

input.shape: torch.Size([3, 10, 26]) RNN输入维度,batch,输出维度 hidden_state.shape: torch.Size([1, 10, 8]) (RNN层数*深度, batch, 隐藏层维度)
tensor([[[ 0.0201,  0.0694, -0.0057,  0.0396, -0.1485, -0.0943,  0.0833,
          -0.0261],
         [-0.0715,  0.0271,  0.0697,  0.0165, -0.2064,  0.0282,  0.0626,
           0.0361],
         [ 0.0348,  0.0584,  0.0725,  0.0626, -0.0822, -0.0583,  0.0071,
          -0.0385],
         [-0.1085,  0.0292,  0.0830,  0.0352, -0.1512, -0.0061, -0.0092,
           0.0076],
         [-0.1154,  0.0359,  0.0869,  0.0132, -0.0918,  0.0375, -0.0293,
          -0.0392],
         [-0.0847, -0.0263,  0.0923,  0.0736, -0.1362, -0.0391,  0.0915,
           0.0118],
         [-0.1154,  0.0359,  0.0869,  0.0132, -0.0918,  0.0375, -0.0293,
          -0.0392],
         [-0.0847, -0.0263,  0.0923,  0.0736, -0.1362, -0.0391,  0.0915,
           0.0118],
         [-0.0847, -0.0263,  0.0923,  0.0736, -0.1362, -0.0391,  0.0915,
           0.0118],
         [-0.0180, -0.0276, -0.0065,  0